In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python

In [1]:
import sys
sys.path.append(r"./python")

import os
import json
from model import *

#根据时间情况修改index和language值
index =  ""
language = "english"
embedding_endpoint_name = "cohere.embed-multilingual-v3"

embedding_type = 'bedrock' if embedding_endpoint_name.find('titan') or embedding_endpoint_name.find('cohere') else 'sagemaker'
embeddings = init_embeddings_bedrock(embedding_endpoint_name)

In [ ]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
import fitz
from PIL import Image
import numpy as np
import base64
from opensearch_multimodel_dataload import add_multimodel_documents

model_name = "anthropic.claude-3-sonnet-20240229-v1:0"
llm = init_model_bedrock(model_name)
text_max_length = 2000

def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError as e:
        return False
    return True

prompt = """
You are a document organizer of manufacture company and your task is to extract useful information from images.Please refer to the format of the content of the previous page to extract text information from the image on the current page.
If the content on this page contains tables or maintenance process, please organize the tables or maintenance process into json format. 
If the content of this page is a table and has no header, use the header of the previous page. 
If the content of this page is a maintenance process and does not specify the specific maintenance object, use the maintenance object on the previous page.

<previous page content>
{context}
</previous page content>
No preface, just output the content directly.
"""

files_path = '../docs/giant/'
# os.mkdir('images/')
files = os.listdir(files_path)
for file in files:
    file_path = files_path + file
    print(file_path)
    # fname = file.split('/')[-1].split('.')[0]
    # print(fname)
    # os.mkdir('images/'+fname)

    doc = fitz.open(file_path)
    previous_page_content = ''
    
    texts = []
    metadatas = []
    images = []
    
    for i in tqdm(range(doc.page_count)):
        if i < 2:
            continue
            
        page = doc.load_page(i)
        pix = page.get_pixmap()
        
        # img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        # img_name = fname + '-' + str(i) + '.jpg'
        # img.save('images/'+fname + '/' + img_name, "JPEG")
        
        imgb64 = base64.b64encode(pix.tobytes()).decode("utf-8")
        model_kwargs = {'image': imgb64,'max_tokens':2048}
        llm.model_kwargs = model_kwargs
        new_prompt = prompt.format(context=previous_page_content)
        response = llm(prompt=new_prompt)
        response = response.strip()
        previous_page_content = response
        
        print('response:',response)
        response_set = set()
        if is_json(response) and response.find('{') >=0:
            response = json.loads(response)
            for data in response:
                if isinstance(response[data], dict):
                    for sub_key in response[data]:
                        text = str(sub_key) + '\n' + str(response[data][sub_key])
                        response_set.add(text)
                else:
                    text = str(data) + '\n' + str(response[data])
                    response_set.add(text)
            response = json.dumps(response)
        else:
            response_list = response.split('\n')
            for data in response_list:
                data = data.strip().replace('null','')
                if len(data) > 2:
                    if is_json(data) and data.find('{') >=0:
                        data = json.loads(data)
                        for sub_key in data:
                            text = str(sub_key) + '\n' + str(data[sub_key])
                            response_set.add(text)
                    else:
                        response_set.add(data)
        for text in response_set:
            texts.append(str(response).replace(': null,',''))
            images.append(imgb64)
            metadata = {}
            metadata['sentence'] = text[:text_max_length] if len(text) > text_max_length else text
            metadata['sources'] = file.split('/')[-1]
            metadata['page'] = i
            metadatas.append(metadata)
    if len(texts) > 0:
        if embedding_type == 'bedrock':
            text_embeddings = embeddings.embed_documents(list([metadata["sentence"] for metadata in metadatas]))
        else:
            text_embeddings = embeddings.embed_documents(list([metadata["sentence"] for metadata in metadatas]),chunk_size=10)

        print('texts len:',len(texts))
        print('metadatas len:',len(metadatas))
        print('embeddings len:',len(text_embeddings))
        print('images len:',len(images))
        print('begin to save in vectore store')

        add_multimodel_documents(
            index,
            texts=texts,
            embeddings=text_embeddings,
            metadatas=metadatas,
            images=images
        )
        print('finish save in vectore store')
